In [1]:
import pandas as pd
import json
import pickle
import duckdb
import unicodedata
conn = duckdb.connect('/srv/data/grela/grela_v0-3.duckdb', read_only=True)

NOUN, VERB, ADJ, PROPN
  AND t.pos IN ('n', 'v', 'a')
AND t.pos IN ('NOUN', 'VERB', 'ADJ', 'PROPN')

In [5]:
query = '''
SELECT t.*, w.*, s.*
FROM tokens t
JOIN works w ON t.grela_id = w.grela_id
JOIN sentences s ON t.sentence_id = s.sentence_id
WHERE w.lagt_provenience IN ('christian', 'pagan')
  AND t.pos IN ('n', 'v', 'a', 'NOUN', 'VERB', 'ADJ', 'PROPN')
  AND (
    (w.not_before > 0 AND w.not_before < 600)
    OR (w.not_after > 0 AND w.not_after < 600)
  )
'''
df = conn.execute(query).fetchdf()
df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,sentence_id,grela_id,token_text,lemma,pos,char_start,char_end,token_id,ref,grela_source,...,author_viaf,title_viaf,date_random,token_count,textsource,sentence_id_1,grela_id_2,position,text,subwork_id
0,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,ἀνεγνωκὼς,ἀναγιγνώσκω,VERB,31,40,346932868,"{""div_section"": ""22""}",lagt,...,NaN,NaN,85.0,6639,glaux,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,311,ὡς τῶν παρ ἡμῖν τις κατὰ τύχην ἀνεγνωκὼς δύο τ...,
1,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,Ἐφόρου,Ἔφορος,NOUN,49,55,346932871,"{""div_section"": ""22""}",lagt,...,NaN,NaN,85.0,6639,glaux,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,311,ὡς τῶν παρ ἡμῖν τις κατὰ τύχην ἀνεγνωκὼς δύο τ...,
2,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,βιβλίων,βιβλίον,NOUN,56,63,346932872,"{""div_section"": ""22""}",lagt,...,NaN,NaN,85.0,6639,glaux,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,311,ὡς τῶν παρ ἡμῖν τις κατὰ τύχην ἀνεγνωκὼς δύο τ...,
3,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,ἀνθρώπους,ἄνθρωπος,NOUN,79,88,346932877,"{""div_section"": ""22""}",lagt,...,NaN,NaN,85.0,6639,glaux,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,311,ὡς τῶν παρ ἡμῖν τις κατὰ τύχην ἀνεγνωκὼς δύο τ...,
4,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,κατέτριβε,κατατρίβω,VERB,89,98,346932878,"{""div_section"": ""22""}",lagt,...,NaN,NaN,85.0,6639,glaux,lagt_tlg0007.tlg101_311,lagt_tlg0007.tlg101,311,ὡς τῶν παρ ἡμῖν τις κατὰ τύχην ἀνεγνωκὼς δύο τ...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10128841,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,κλίνην,κλίνη,NOUN,21,27,382371991,"{""oga_cts"": ""6"", ""oga_tid"": ""t_11615""}",lagt,...,NaN,NaN,565.0,99860,oga,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,661,"ἰδοὺ βάλλω αὐτὴν εἰς κλίνην, καὶ τοὺς μοιχεύον...",
10128842,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,μοιχεύοντας,μοιχεύω,VERB,38,49,382371995,"{""oga_cts"": ""6"", ""oga_tid"": ""t_11619""}",lagt,...,NaN,NaN,565.0,99860,oga,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,661,"ἰδοὺ βάλλω αὐτὴν εἰς κλίνην, καὶ τοὺς μοιχεύον...",
10128843,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,θλῖψιν,θλῖψις,NOUN,64,70,382371999,"{""oga_cts"": ""6"", ""oga_tid"": ""t_11623""}",lagt,...,NaN,NaN,565.0,99860,oga,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,661,"ἰδοὺ βάλλω αὐτὴν εἰς κλίνην, καὶ τοὺς μοιχεύον...",
10128844,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,μεγάλην,μέγας,ADJ,71,78,382372000,"{""oga_cts"": ""6"", ""oga_tid"": ""t_11624""}",lagt,...,NaN,NaN,565.0,99860,oga,lagt_tlg4102.tlgX02_661,lagt_tlg4102.tlgX02,661,"ἰδοὺ βάλλω αὐτὴν εἰς κλίνην, καὶ τοὺς μοιχεύον...",


In [15]:
# unifying the postags
df['pos'] = df['pos'].replace({'v': 'VERB', 'n': 'NOUN', 'a': 'ADJ'})
df['pos'].unique()

array(['VERB', 'NOUN', 'ADJ', 'PROPN'], dtype=object)

In [16]:
grouped = (
    df.groupby('sentence_id').agg(
        lamma_sentence=pd.NamedAgg(column='lemma', aggfunc=lambda x: ' '.join(x)),
        token_ids=pd.NamedAgg(column='token_id', aggfunc=list),
        title=pd.NamedAgg(column='title', aggfunc='first'),
        author=pd.NamedAgg(column='author', aggfunc='first'),
        lagt_provenience=pd.NamedAgg(column='lagt_provenience', aggfunc='first'),
        not_before=pd.NamedAgg(column='not_before', aggfunc='first'),
        not_after=pd.NamedAgg(column='not_after', aggfunc='first'),
        sentence_text=pd.NamedAgg(column='text', aggfunc='first'),
        lemma_list=pd.NamedAgg(column='lemma', aggfunc=list),
        token_list=pd.NamedAgg(column='token_text', aggfunc=list),
        pos_list=pd.NamedAgg(column='pos', aggfunc=list),
        char_start_list=pd.NamedAgg(column='char_start', aggfunc=list),
        char_end_list=pd.NamedAgg(column='char_end', aggfunc=list),
    ).reset_index()
)

In [17]:
grouped['tokens'] = [
    json.dumps([
        {
            'lemma': lemma,
            'token': token,
            'pos': pos,
            'char_start': char_start,
            'char_end': char_end
        }
        for lemma, token, pos, char_start, char_end in zip(row['lemma_list'], row['token_list'], row['pos_list'], row['char_start_list'], row['char_end_list'])
    ], ensure_ascii=False)
    for _, row in grouped.iterrows()
]

grouped = grouped.drop(columns=['lemma_list', 'token_list', 'pos_list', 'char_start_list', 'char_end_list'])
grouped.head()

,sentence_id,lamma_sentence,token_ids,title,author,lagt_provenience,not_before,not_after,sentence_text,tokens
0,lagt_ogl0001.ogl001_0,πινυτός ἀντίγραφον θαυμάζω ἀποδέχομαι διονύσιος,"[365056825, 365056826, 365056828, 365056831, 3...",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,"πρὸς ἣν, ὁ Πινυτὸς ἀντιγράφων, θαυμάζει μὲν κ...","[{""lemma"": ""πινυτός"", ""token"": ""Πινυτὸς"", ""pos..."
1,lagt_ogl0001.ogl001_1,ἀντιπαρακαλέω σκληρός μεταδίδωμι τροφή τέλειος...,"[365056835, 365056837, 365056840, 365056841, 3...",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,ἀντιπαρακαλεῖ δὲ στεῤῥοτέρας ἤδη ποτὲ μεταδιδό...,"[{""lemma"": ""ἀντιπαρακαλέω"", ""token"": ""ἀντιπαρα..."
2,lagt_ogl0001.ogl001_2,ἐπιστολή πινυτός πίστις εὐσέβεια φροντίς ὑπήκο...,"[365056868, 365056872, 365056875, 365056876, 3...",De Epistola Pinyti ad Dionysium,Pinytus,christian,101.0,200.0,δι᾿ ἧς ἐπιστολῆς καὶ ἡ τοῦ Πινυτοῦ περὶ τὴν πί...,"[{""lemma"": ""ἐπιστολή"", ""token"": ""ἐπιστολῆς"", ""..."
3,lagt_tlg0004.tlg001_0,φιλοσοφία ἔργον ἔνιοι φημί βάρβαρος ἄρχω,"[345704070, 345704071, 345704072, 345704073, 3...",Βίοι καὶ γνῶμαι τῶν ἐν φιλοσοφίᾳ εὐδοκιμησάντων,Diogenes Laertius,pagan,222.0,235.0,τὸ τῆς φιλοσοφίας ἔργον ἔνιοί φασιν ἀπὸ βαρβάρ...,"[{""lemma"": ""φιλοσοφία"", ""token"": ""φιλοσοφίας"",..."
4,lagt_tlg0004.tlg001_1,γίγνομαι Πέρσης Μάγος Βαβυλώνιος Ἀσσύριος Χαλδ...,"[345704078, 345704082, 345704083, 345704087, 3...",Βίοι καὶ γνῶμαι τῶν ἐν φιλοσοφίᾳ εὐδοκιμησάντων,Diogenes Laertius,pagan,222.0,235.0,"γεγενῆσθαι γὰρ παρὰ μὲν Πέρσαις Μάγους, παρὰ δ...","[{""lemma"": ""γίγνομαι"", ""token"": ""γεγενῆσθαι"", ..."


In [18]:
# make the tokens machine-readible again...
grouped["tokens"] = grouped["tokens"].apply(eval)


# normalize token and lemma text within tokens column...
# If you want to keep diaeresis, include "\u0308".
KEEP_COMBINING = set()  # e.g., set(["\u0308"]) to keep diaeresis

def normalize_greek(text: str, *, strip_diacritics: bool = True) -> str:
    if not isinstance(text, str):
        return text
    # 1) Decompose so diacritics become combining marks
    s = unicodedata.normalize("NFD", text)

    # 2) Unify variant apostrophes/koronis to a single mark (’)
    #    This helps reduce visual variants in Greek texts
    s = (
        s.replace("\u1FBD", "’")  # Greek Koronis
        .replace("\u02BC", "’")  # Modifier Letter Apostrophe
        .replace("\u2019", "’")  # Right single quotation mark
        .replace("\u00B4", "’")  # Spacing acute accent (rarely used)
        .replace("'", "’")       # ASCII apostrophe
    )

    # 3) Optionally strip all combining marks (accents, breathings, subscripts)
    if strip_diacritics:
        s = "".join(
            ch for ch in s
            if not (unicodedata.category(ch) == "Mn" and ch not in KEEP_COMBINING)
        )

    # 4) Recompose
    return unicodedata.normalize("NFC", s)

def norm_tokens_lemmata(tokens, *, strip_diacritics: bool = True):
    out = []
    for t in tokens or []:
        if not isinstance(t, dict):
            continue
        d = dict(t)  # avoid mutating the original
        if "lemma" in d and isinstance(d["lemma"], str):
            d["lemma"] = normalize_greek(d["lemma"], strip_diacritics=strip_diacritics)
        if "token" in d and isinstance(d["token"], str):
            d["token"] = normalize_greek(d["token"], strip_diacritics=strip_diacritics)
        out.append(d)
    return out


# apply the normalization
grouped["tokens"] = grouped["tokens"].apply(lambda toks: norm_tokens_lemmata(toks, strip_diacritics=False))


In [19]:
def make_subcorpora(not_before, not_after, lagt_provenience):
    if lagt_provenience == 'christian':
        if not_before >= 0 and not_after <= 300:
            return "christian_0_300"
        elif not_before >= 300 and not_after <= 600:
            return "christian_300_600"
    elif lagt_provenience == 'pagan':
        if not_before >= 0 and not_after <= 300:
            return "pagan_0_300"
        elif not_before >= 300 and not_after <= 600:
            return "pagan_300_600"
    return None

grouped["enemy_subcorpus"] = grouped.apply(
    lambda row: make_subcorpora(row["not_before"], row["not_after"], row["lagt_provenience"]), axis=1
)

In [21]:
# store into your personal location:
grouped.to_pickle('../data/large-data/grouped_df.pkl')
# store into our shared data directory
grouped.to_pickle('/srv/data/enemy-christ/large-data/grouped_df.pkl')

In [2]:
### later tests...
grouped = pd.read_pickle('/srv/data/enemy-christ/large-data/grouped_df.pkl')

In [22]:
# Python
def detect_target(tokens, target):
    return any(t.get("lemma") == target for t in tokens)

target = 'ἐχθρός'


enemy_sentences_all = grouped[grouped["tokens"].apply(lambda x: detect_target(x, target))]

In [23]:
len(enemy_sentences_all)

3204